In [6]:
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import unittest
%matplotlib inline

Library

In [7]:
def assigning_edges(Grid: 'Networkx Edgeless Graph', grid_map: 'numpy array',dimensions: (int,int)):
    for row in range(dimensions[0]):
        for col in range(dimensions[1]):
            directions = {'up': True,'down': True,'left': True,'right': True, 'up-left': True, 'up-right': True, 'down-left': True, 'down-right': True}
            #direction removal checks
            if row == 0:
                directions['up'] = False
                directions['up-left'] = False
                directions['up-right'] = False
            if row == dimensions[0]-1:
                directions['down'] = False
                directions['down-left'] = False
                directions['down-right'] = False
            if col == 0:
                directions['left'] = False
                directions['up-left'] = False
                directions['down-left'] = False
            if col == dimensions[1]-1:
                directions['right'] = False
                directions['up-right'] = False
                directions['down-right'] = False

            for direction, tf in directions.items():
                if tf:
                    if directions=='up':
                        Grid.add_edge(grid_map[row,col], grid_map[row-1,col])
                    elif direction=='right':
                        Grid.add_edge(grid_map[row,col], grid_map[row,col+1])
                    elif direction=='down':
                        Grid.add_edge(grid_map[row,col], grid_map[row+1,col])
                    elif direction=='left':
                        Grid.add_edge(grid_map[row,col], grid_map[row,col-1])
                    elif direction=='up-left':
                         Grid.add_edge(grid_map[row,col], grid_map[row-1,col-1])
                    elif direction=='up-right':
                         Grid.add_edge(grid_map[row,col], grid_map[row-1,col+1])
                    elif direction=='down-left':
                         Grid.add_edge(grid_map[row,col], grid_map[row+1,col-1])
                    elif direction=='down-right':
                         Grid.add_edge(grid_map[row,col], grid_map[row+1,col+1])
    return Grid
def generate_grid(row: int, height: int) -> 'Networkx Graph':
    dimensions = (row,height)
    list_grid_map = np.arange(row*height)
    grid_map = list_grid_map.reshape(row,height)
    Grid = nx.Graph()
    Grid.add_nodes_from(list_grid_map)
    
    Grid = assigning_edges(Grid, grid_map, dimensions)
    return Grid

Map class

In [10]:
class Map:
    def __init__(self, row=3, height=3):
        self.grid = generate_grid(row,height)
        
        self.dimensions = (row,height)
        
        self.grid_map = np.arange(row*height)
        self.grid_map.reshape(row,height)
        
        
        self.obstacles = {}
    def get_grid(self):
        return self.grid
    def get_dimensions(self):
        return self.dimensions
    def get_grid_map(self):
        return self.grid_map
    def get_node_options(self, node):
        return list(self.grid[node])
    
    def add_node(self, edges: [int]):
        new_node = self.dimensions[0] * self.dimensions[1]
        self.grid.add_node(new_node)
        for i in edges:
            self.grid.add_edge(new_node, i)
        
    def add_edge(self, edge):
        self.grid.add_edge(*edge)
        
    def add_obstacle(self, node):
        self.obstacles[node] = list(self.grid[node])
        self.remove_node(node)
        
    def remove_node(self, node):
        self.grid.remove_node(node)
        
    def remove_edge(self, edge):
        self.grid.remove_edge(*edge)

    def remove_obstacle(self, node):
        self.grid.add_node(node)
        for i in self.obstacles[node]:
            self.add_edge( (node, i) )
        self.obstacles.pop(node, None)
            
    def raw_draw(self):
        nx.draw(self.grid, with_labels=True, font_weight='bold')

User class

In [9]:
class User:
    def __init__(self):
        #contains map
        #contains path
        #contains path cost
        #contains current node
        pass
    '''
    list of methods:
        add to path and update map
        remove from path and update map
        return path cost
        return map
        return current node
        return whether reached end destination or not
        return whether stuck (no way to move)t
    '''

In [12]:
class User:
    def __init__(self, Map: 'map', start=0, waypoints=[int]):
        self.map = Map
        self.path = [self.map.get_start()]
        self.path_cost = 0
        self.current_node = self.map.get_start()
        
        self.start = start
        self.waypoints = waypoints
    def get_grid(self):
        return self.grid
    def get_path(self):
        return self.path
    def get_cost(self):
        return self.cost
    def get_current_node(self):
        return self.current_node
    def get_start(self):
        return self.start
    def get_waypoints(self):
        return self.waypoints
    
    def get_current_options(self):
        return self.map.get_node_options(self.current_node)
    def get_traversed(self):
        return [node for node in self.waypoints if node in self.path]
    
    def travel_node(self, node):
        self.path.append(node)
        self.path_cost += 1
        
    def reverse_travel(self):
        del self.path[-1]
        self.path_cost -= 1
    
    
    def reached_end(self):
        for node in self.waypoints:
            if node not in self.path:
                return False
        return True
    def is_stuck(self):
        return len(self.get_current_options())==0